# Web scraping

## Co je web scraping?
- Strojové čtení nestrukturovaných dat z webových stránek

## Co není web scraping?
- Stahování dat přes API
- Stahování strukturovaných dat (JSON, CSV,...)
- Crawling - procházení a indexování celé webové stránky pomocí jejích vnitřních hypertextových odkazů


## Příklady web scrapingu

### Analýza cen na českých e-shopech


<div>
    <img src="static/hlidacshopu.png", width="50%"/>
</div>

https://medium.com/@jakubbalada/black-friday-2019-s-hl%C3%ADda%C4%8Dem-shop%C5%AF-9a3ddd352a8c

### Jak hodnotila filmy Mirka Spáčilová


<div>
<img src="static/spacilova.png", width="50%"/>
</div>

https://www.michalblaha.cz/2017/10/filmova-kriticka-mirka-spacilova-v-cislech/

## Etika web scrapingu

- Než začneš s web scrapingem, podívej se, jestli stránka nenabízí strukturovaná data ke stažení nebo neposkytuje API. (Jak pracovat s API si ukážeme někdy jindy.)
    - **Příklady:** 
    - https://data.gov.cz/datov%C3%A9-sady?poskytovatel=%C4%8Cesk%C3%BD%20statistick%C3%BD%20%C3%BA%C5%99ad
    - https://www.ncdc.noaa.gov/data-access
    - https://www.mapakriminality.cz/data/
    - http://opendata.praha.eu/dataset/meteostanice-chmi-api
- Zjisti si, jaká máš práva k datům, nepublikuj získaná data nelegálně
- Přistupuj ke stránce k rozumné míře, nesnažíš se stránku shodit, ale získat data :-) 

## Z čeho se skládá webová stránka
- **HTML** (HyperText Markup Language): strukturovaný obsah stránky (text a obrázky)
- **CSS** (Cascading Style Sheets): úprava vzhledu stránky
- **JavaScript**: interaktivita obsahu a vzhledu stránky

### HTML 
<div>
    <img src="static/html.png", width="80%">
</div>

- Je tvořen HTML značkami / tagy, např.  ``<img>``
- Většina HTML tagů je párová, např. ``<h2>`` a ``</h2>``
- Tagy mohou mít atributy, které dále specifikují, co a jak bude tag zobrazovat
- Atribut class se obvykle používá k stylování stránky a často podle něj můžeme při webscrapingu odlišit různé části stránky

### CSS

- Popisuje způsob zobrazení html elementů
- Obsahuje 2 části: selektor elementu a blok deklarace:

```
p.error {
  color: red;
}
```


## Pandas: read_html()

Pokud nám stačí stáhnout tabulky z webu, můžeme použít knihovnu [Pandas](https://pandas.pydata.org/pandas-docs/version/0.25/reference/api/pandas.read_html.html):

In [1]:
import pandas as pd

Převedeme tabulky ze stránky Wikipedie [Seznam států světa podle spotřeby alkoholu](https://cs.wikipedia.org/wiki/Seznam_st%C3%A1t%C5%AF_sv%C4%9Bta_podle_spot%C5%99eby_alkoholu) do dataframů pomocí funce `read_html()`:

In [2]:
tables = pd.read_html('https://cs.wikipedia.org/wiki/Seznam_st%C3%A1t%C5%AF_sv%C4%9Bta_podle_spot%C5%99eby_alkoholu')

Výsledkem je seznam dataframů:

In [3]:
type(tables)

list

In [4]:
len(tables)

2

In [5]:
tables[0].head()

,stát,evidováno,neevidováno,celkem,pivo,víno,destiláty,ostatní
0,Česko,14.97,1.48,16.45,8.51,2.33,3.59,0.39
1,Maďarsko,12.27,4.00,16.27,4.42,4.94,3.02,0.14
2,Rusko,11.03,4.73,15.76,3.65,0.10,6.88,0.34
3,Ukrajina,8.10,7.50,15.60,2.69,0.58,5.21,0.02
4,Estonsko,13.77,1.80,15.57,5.53,1.09,9.19,0.43


In [6]:
tables[1].head()

,Pořadí,Stát,Spotřeba v litrech,Rok
0,1,Francie,12.6,2011
1,2,Rakousko,12.2,2009
2,3,Estonsko,12.0,2011
3,4,Německo,11.7,2009
4,5,Irsko,11.6,2011


**Varování:** Svět není ideální (proto se v něm taky tolik pije) a ne vždy se podaří tabulky pomocí `read_html()` získat. Příkaz nemusí tabulku vůbec "vidět", nebo si nedokáže poradit s její strukturou.

### Cvičení

Získejte tabulku aktuálních ekonomických údajů ze stránek Českého statistického úřadu, `https://www.czso.cz/csu/czso/aktualniinformace`.

**Nápověda**: Pokud uvidíš v dataframu ošklivé znaky, zkus specifikovat `encoding='utf-8'`

Pokud data, která hledáme, nejsou na webu ve formě tabulky, musíme přikročit k „drastičtějším“ nástrojům :-)

## BeautifulSoup

Knihovna [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) se používá k získávání dat z HTML a XML souborů. Pracuje s různými parsery, které analyzují HTML soubory, a umožnuje vybrat požadované HTML elementy a pracovat s nimi. 

Pod čarou: Kdybys měl/a větší projekt a potřeboval/a projít třeba několik desítek e-shopů, podívej se na knihovnu [Scrapy](http://docs.scrapy.org/en/latest/intro/overview.html), která je pro podobné úkoly vhodnější.


In [7]:
from bs4 import BeautifulSoup

Nejprve si načteme vzorový html dokument ze souboru:

In [8]:
with open("static/html_doc.html", "r") as f:
    html_doc = f.read()

Vytvoříme objekt typu `BeautifulSoup`. První argument jsou HTML data, druhý slouží k specifikaci parseru.

In [9]:
soup = BeautifulSoup(html_doc, 'html.parser')

Pomocí metody `prettify()` můžeme vytisknout hezky zformátované html:

In [10]:
#print(soup.prettify())

Po objektu typu `BeautifulSoup` se můžeme pohybovat pomocí tagů.  
První element typu title:

In [11]:
soup.title

<title>PyData Prague | pydata.cz</title>

První element typu `h1`:

In [12]:
soup.h1

<h1><a href="https://pydata.cz/">pydata.cz</a></h1>

Jméno elementu typu `h1`:

In [13]:
soup.h1.name

'h1'

Jméno nadřazeného elementu (atributy `parent` a `name`): 

In [14]:
soup.h1.parent.name

'div'

Data uvnitř elementu `h1` (atribut [`string`](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#string))

In [15]:
soup.h1.string

'pydata.cz'

První element typu `h2`:

In [16]:
soup.h2

<h2 id="code-of-conduct">Code of Conduct</h2>

Id elementu `h2`:

In [17]:
soup.h2['id']

'code-of-conduct'

Můžeme najít i všechny tagy daného typu, např `find_all('a')` najde všechny odkazy:

In [18]:
links = soup.find_all('a')
links

[<a href="https://pydata.cz/">pydata.cz</a>,
 <a class="pydata" href="https://pydata.org/">PyData</a>,
 <a href="https://numfocus.org/">NumFOCUS</a>,
 <a class="pydata" href="https://pydata.org/code-of-conduct/">pydata.org/code-of-conduct/</a>]

Výsledky hledání můžeme procházet:

In [19]:
links[0]['href']

'https://pydata.cz/'

In [20]:
links[0].text

'pydata.cz'

Nemusíme hledat jen podle jména tagu, ale i podle jejich atributů:

In [21]:
soup.find(id="pydata-prague")

<h1 id="pydata-prague">PyData Prague</h1>

Pokud hledáme podle třídy, memůžeme použít ``soup.find(class="pydata")``, protože ``class`` je v Pythonu klíčové slovo. Musíme použít ``class_``

In [22]:
soup.find(class_="pydata")

<a class="pydata" href="https://pydata.org/">PyData</a>

``class`` můžeme taky napsat jako klíč v parametru ``attrs``:

In [23]:
soup.find(attrs={'class':'pydata'})

<a class="pydata" href="https://pydata.org/">PyData</a>

### Cvičení


Vyberte odstavec s id ``description``.

Vyberte všechny odkazy s třídou ``pydata``

## Requests

Knihovna [requests](https://requests.readthedocs.io/en/master/) je určená pro HTTP dotazy. V našem případě ji budeme používat pro získání textu webové stránky.


In [24]:
import requests

In [25]:
r = requests.get('https://pydata.cz/')

Můžeme zkontrolovat návratový stav:

In [26]:
r.status_code

200

[Stavové kódy](https://cs.wikipedia.org/wiki/Stavov%C3%A9_k%C3%B3dy_HTTP) se dělí do 5 skupin:
- 1xx – informační odpověď
- 2xx – úspěch
- 3xx – přesměrování
- 4xx – chyba klienta
- 5xx – chyba serveru

Metoda text ukáže zdrojový kód stránky:

In [27]:
#print(r.text)

## BeautifulSoup podruhé: CSS selektory

S knihovnou BeautifulSoup můžeme [vyhledávat CSS selektory](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#css-selectors) pomocí funkcí `select` (ukáže všechny prvky) a `select_one` (najde první prvek).

In [28]:
r = requests.get('https://dokina.tiscali.cz/program-kin')

In [29]:
soup = BeautifulSoup(r.text)

Zkusíme najít všechny tagu s třídou `title` a několik z nich si vypíšeme:  
(Kvůli přehlednosti nevypisujeme úplně všechny)

In [30]:
titles = soup.select('.title') 
titles[:6]

[<h4 class="title mb-0">3Bobule</h4>,
 <h4 class="title mb-0">Tiché místo: Část II</h4>,
 <h4 class="title mb-0">V síti</h4>,
 <h4 class="title mb-0">Princezna zakletá v čase</h4>,
 <h4 class="title mb-0">Bloodshot</h4>,
 <h3 class="title mb-0">
 <a data-ga-action="cinema-detail" data-ga-category="program-kin" href="/ponrepo-2836" title="Profil kina">Ponrepo</a>
 <a class="favourite-toggler" data-ga-action="toggle-favorite-cinema" data-ga-category="program-kin" data-id="2836" href="#" title="Přidat do oblíbených"></a>
 </h3>]

Najdeme všechny tagy typu  `<h3 class="title mb-0">` a podíváme se na první z nich:

In [31]:
mb0_titles = soup.select('h3.title.mb-0')
mb0_titles[0]

<h3 class="title mb-0">
<a data-ga-action="cinema-detail" data-ga-category="program-kin" href="/ponrepo-2836" title="Profil kina">Ponrepo</a>
<a class="favourite-toggler" data-ga-action="toggle-favorite-cinema" data-ga-category="program-kin" data-id="2836" href="#" title="Přidat do oblíbených"></a>
</h3>

Najdeme všechny tagy s třídou `title` uvnitř tagů s třídou `movie-item`

In [32]:
soup.select('.movie-item .title') 

[<h4 class="title mb-0">3Bobule</h4>,
 <h4 class="title mb-0">Tiché místo: Část II</h4>,
 <h4 class="title mb-0">V síti</h4>,
 <h4 class="title mb-0">Princezna zakletá v čase</h4>,
 <h4 class="title mb-0">Bloodshot</h4>,
 <h4 class="title mb-0">Noční ostraha</h4>,
 <h4 class="title mb-0">Karel, já a ty</h4>,
 <h4 class="title mb-0">FREM</h4>,
 <h4 class="title mb-0">Malé ženy</h4>,
 <h4 class="title mb-0">Bláznivý Petříček</h4>,
 <h4 class="title mb-0">Srdcová královna</h4>,
 <h4 class="title mb-0">Disco</h4>,
 <h4 class="title mb-0">Junction 48</h4>,
 <h4 class="title mb-0">Mlsné medvědí příběhy</h4>,
 <h4 class="title mb-0">Naděje</h4>,
 <h4 class="title mb-0">Občan K.</h4>,
 <h4 class="title mb-0">Vysoká dívka</h4>,
 <h4 class="title mb-0">Zákeřná Ebola</h4>,
 <h4 class="title mb-0">Šarlatán</h4>,
 <h4 class="title mb-0">Žaluji!</h4>,
 <h4 class="title mb-0">Žáby bez jazyka</h4>]

## Nástroje v prohlížeči 

Než začneme procházet stránku pomocí Pythonu, je dobré podívat se na její strukturu přímo v prohlížeči.

### Zdrojový kód stránky

<br />

<div>
<img src="static/FF_pagesource.png" width="30%"align="left"/>
<img src="static/chrome_pagesource.png" align="left"/> 
</div>

### Prozkoumat prvek / Inspect

<br />

<div>
<img src="static/FF_inspect.png" width="30%"align="left"/>
    <b>Pozor:</b> Zrojový kód vybraného prvku nemusí odpovídat kódu, který stáhneš pomocí Pythonu. Během zobrazení v prohlížeči mohl být změněn JavaScriptem.
<img src="static/Inspect.png" align="left"/> 
</div>

### Nástroje pro vývojáře

<br />

<div>
<img src="static/FF_dev.png" />
<img src="static/chrome_dev.png" /> 
</div>


**Poznámka:** Pokud v záložce Network nic nevidíš, zkus obnovit stránku.

## Příklad z (ne)života

**Extrakce dat o 20 nejlepších filmech podle ČSFD**

Chceme vytvořit tabulku s následujícími sloupci:

- Název filmu
- Průměrné hodnocení
- Země původu
- Rok uvedení
- Délka filmu
- Režisér

*Na scraping není univerzální návod. Obvykle metodou pokus-omyl zkoušíme, co bude fungovat. Pak přijde aktualizace stránek a můžeme začít znovu...*

In [33]:
url = 'https://www.csfd.cz/zebricky/nejlepsi-filmy/'
# stránka odmítá GET requesty bez identifikace User-Agenta:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')

### Úkol:
Podívej se do zdrojového kódu stránky. Jaké prvky budeme na stránce hledat? A kde najdeme informace o režisérovi?

In [34]:
soup.select('.film')

[<td class="film" id="chart-2294"><a href="/film/2294-vykoupeni-z-veznice-shawshank/">Vykoupení z věznice Shawshank</a> <span class="film-year" dir="ltr">(1994)</span></td>,
 <td class="film" id="chart-10135"><a href="/film/10135-forrest-gump/">Forrest Gump</a> <span class="film-year" dir="ltr">(1994)</span></td>,
 <td class="film" id="chart-2292"><a href="/film/2292-zelena-mile/">Zelená míle</a> <span class="film-year" dir="ltr">(1999)</span></td>,
 <td class="film" id="chart-2982"><a href="/film/2982-prelet-nad-kukaccim-hnizdem/">Přelet nad kukaččím hnízdem</a> <span class="film-year" dir="ltr">(1975)</span></td>,
 <td class="film" id="chart-2671"><a href="/film/2671-sedm/">Sedm</a> <span class="film-year" dir="ltr">(1995)</span></td>,
 <td class="film" id="chart-8653"><a href="/film/8653-schindleruv-seznam/">Schindlerův seznam</a> <span class="film-year" dir="ltr">(1993)</span></td>,
 <td class="film" id="chart-1644"><a href="/film/1644-kmotr/">Kmotr</a> <span class="film-year" dir=

In [35]:
film_odkazy = soup.select('.film a')
film_odkazy

[<a href="/film/2294-vykoupeni-z-veznice-shawshank/">Vykoupení z věznice Shawshank</a>,
 <a href="/film/10135-forrest-gump/">Forrest Gump</a>,
 <a href="/film/2292-zelena-mile/">Zelená míle</a>,
 <a href="/film/2982-prelet-nad-kukaccim-hnizdem/">Přelet nad kukaččím hnízdem</a>,
 <a href="/film/2671-sedm/">Sedm</a>,
 <a href="/film/8653-schindleruv-seznam/">Schindlerův seznam</a>,
 <a href="/film/1644-kmotr/">Kmotr</a>,
 <a href="/film/306731-nedotknutelni/">Nedotknutelní</a>,
 <a href="/film/6178-dvanact-rozhnevanych-muzu/">Dvanáct rozhněvaných mužů</a>,
 <a href="/film/4570-pelisky/">Pelíšky</a>,
 <a href="/film/1645-kmotr-ii/">Kmotr II</a>,
 <a href="/film/1248-terminator-2-den-zuctovani/">Terminátor 2: Den zúčtování</a>,
 <a href="/film/8852-pulp-fiction-historky-z-podsveti/">Pulp Fiction: Historky z podsvětí</a>,
 <a href="/film/5911-tenkrat-na-zapade/">Tenkrát na Západě</a>,
 <a href="/film/2356-mlceni-jehnatek/">Mlčení jehňátek</a>,
 <a href="/film/4711-pan-prstenu-spolecenstvo-p

In [36]:
nazvy = [film.text for film in film_odkazy]
nazvy

['Vykoupení z věznice Shawshank',
 'Forrest Gump',
 'Zelená míle',
 'Přelet nad kukaččím hnízdem',
 'Sedm',
 'Schindlerův seznam',
 'Kmotr',
 'Nedotknutelní',
 'Dvanáct rozhněvaných mužů',
 'Pelíšky',
 'Kmotr II',
 'Terminátor 2: Den zúčtování',
 'Pulp Fiction: Historky z podsvětí',
 'Tenkrát na Západě',
 'Mlčení jehňátek',
 'Pán prstenů: Společenstvo Prstenu',
 'Pán prstenů: Návrat krále',
 'Gran Torino',
 'Temný rytíř',
 "Le Mans '66"]

In [37]:
soup.select('.average')

[<td class="average">95,3%</td>,
 <td class="average">94,5%</td>,
 <td class="average">92,8%</td>,
 <td class="average">92,5%</td>,
 <td class="average">92,4%</td>,
 <td class="average">92,3%</td>,
 <td class="average">91,8%</td>,
 <td class="average">91,3%</td>,
 <td class="average">91,4%</td>,
 <td class="average">91,2%</td>,
 <td class="average">90,8%</td>,
 <td class="average">90,8%</td>,
 <td class="average">90,7%</td>,
 <td class="average">90,6%</td>,
 <td class="average">90,5%</td>,
 <td class="average">90,4%</td>,
 <td class="average">90,3%</td>,
 <td class="average">90,3%</td>,
 <td class="average">90,2%</td>,
 <td class="average">90,2%</td>]

In [38]:
hodnoceni = [aver.text for aver in soup.select('.average')]
hodnoceni

['95,3%',
 '94,5%',
 '92,8%',
 '92,5%',
 '92,4%',
 '92,3%',
 '91,8%',
 '91,3%',
 '91,4%',
 '91,2%',
 '90,8%',
 '90,8%',
 '90,7%',
 '90,6%',
 '90,5%',
 '90,4%',
 '90,3%',
 '90,3%',
 '90,2%',
 '90,2%']

In [39]:
odkazy = ['https://www.csfd.cz' + film['href'] for film in film_odkazy]
odkazy

['https://www.csfd.cz/film/2294-vykoupeni-z-veznice-shawshank/',
 'https://www.csfd.cz/film/10135-forrest-gump/',
 'https://www.csfd.cz/film/2292-zelena-mile/',
 'https://www.csfd.cz/film/2982-prelet-nad-kukaccim-hnizdem/',
 'https://www.csfd.cz/film/2671-sedm/',
 'https://www.csfd.cz/film/8653-schindleruv-seznam/',
 'https://www.csfd.cz/film/1644-kmotr/',
 'https://www.csfd.cz/film/306731-nedotknutelni/',
 'https://www.csfd.cz/film/6178-dvanact-rozhnevanych-muzu/',
 'https://www.csfd.cz/film/4570-pelisky/',
 'https://www.csfd.cz/film/1645-kmotr-ii/',
 'https://www.csfd.cz/film/1248-terminator-2-den-zuctovani/',
 'https://www.csfd.cz/film/8852-pulp-fiction-historky-z-podsveti/',
 'https://www.csfd.cz/film/5911-tenkrat-na-zapade/',
 'https://www.csfd.cz/film/2356-mlceni-jehnatek/',
 'https://www.csfd.cz/film/4711-pan-prstenu-spolecenstvo-prstenu/',
 'https://www.csfd.cz/film/4712-pan-prstenu-navrat-krale/',
 'https://www.csfd.cz/film/240479-gran-torino/',
 'https://www.csfd.cz/film/2237

In [40]:
r = requests.get(odkazy[0], headers=headers)
film_soup = BeautifulSoup(r.text, 'html.parser')

In [41]:
film_soup.select_one('.origin').text

'USA, 1994, 142 min'

In [42]:
film_soup.find('span', {'itemprop': 'director'}) 

<span data-truncate="60" itemprop="director">
<a href="/tvurce/2869-frank-darabont/">Frank Darabont</a>
</span>

In [43]:
film_soup.find('span', {'itemprop': 'director'}).a.text

'Frank Darabont'

In [44]:
from time import sleep
import re 

Poznámka pod čarou: `re` je modul standardní knihovny pro práci s řetězci pomocí regulárních výrazů. V následujícím kódu ji použijeme pro rozdělení řetězce podle dvou různých znaků.

In [45]:
zeme = []
roky = []
delky = []
reziseri = []
for odkaz in odkazy:
    r = requests.get(odkaz, headers=headers)
    film_soup = BeautifulSoup(r.text, 'html.parser')
    txt = film_soup.select_one('.origin').text
    puvod, rok, cas, *zbytek = re.split(',|\(', txt) 
    # potřebujeme vyřešit případy typu 166 min (Director's cut: 175 min, Alternativní: 152 min),
    # proto dělíme text podle znaků "," a "("
    reziser = film_soup.find('span', {'itemprop': 'director'}).a.text
    zeme.append(puvod)
    roky.append(rok)
    delky.append(cas)
    reziseri.append(reziser)
    sleep(1) # chováme se lidsky :-)
    

In [46]:
filmy_df = pd.DataFrame(zip(nazvy, zeme, roky, reziseri, delky, hodnoceni), 
                        columns=['Název', 'Původ', 'Rok', 'Režie', 'Délka', 'ČSFD'])
filmy_df

,Název,Původ,Rok,Režie,Délka,ČSFD
0,Vykoupení z věznice Shawshank,USA,1994,Frank Darabont,142 min,"95,3%"
1,Forrest Gump,USA,1994,Robert Zemeckis,142 min,"94,5%"
2,Zelená míle,USA,1999,Frank Darabont,188 min,"92,8%"
3,Přelet nad kukaččím hnízdem,USA,1975,Miloš Forman,133 min,"92,5%"
4,Sedm,USA,1995,David Fincher,127 min,"92,4%"
5,Schindlerův seznam,USA,1993,Steven Spielberg,195 min,"92,3%"
6,Kmotr,USA,1972,Francis Ford Coppola,175 min,"91,8%"
7,Nedotknutelní,Francie,2011,Eric Toledano,112 min,"91,3%"
8,Dvanáct rozhněvaných mužů,USA,1957,Sidney Lumet,96 min,"91,4%"
9,Pelíšky,Česko,1999,Jan Hřebejk,116 min,"91,2%"


## Jak na JavaScript

Pokud webová stránka používá JavaScript k generování obsahu, máme dvě možnosti:

1. Pochopit, co JavaScript dělá, a zařídit se podle toho
2. Chovat se jako browser

### Příklad
Výsledky závodu Big's Backyard Ultra, [`https://my.raceresult.com/139372/#0_2C3B48`](https://my.raceresult.com/139372/#0_2C3B48)


<div>
  <img src="static/bigdog.png" width="80%" align="left">
</div>

In [47]:
r = requests.get('https://my.raceresult.com/139372/#0_2C3B48')

Zkusíme najít v textu stránky slovo Gavin.

In [48]:
r.text.find("Gavin")

-1

Nic. Kde se stala chyba? Vytiskneme si zdrojový kód stránky... a tabulka nikde!

In [49]:
print(r.text)

<!DOCTYPE html>
<html>
<head>
	<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
	<meta http-equiv="content-language" content="">
	<meta name="google" content="notranslate" />
	<meta name="KEYWORDS" content="Big's Backyard Ultra,Ergebnisse,Ergebnis,Result,Results,Anmeldung,Online-Anmeldung,Registration,Urkunden,Urkunde">
	<meta name="description" content="Big's Backyard Ultra, 10/19/2019-10/25/2019, Bell Buckle">
	<meta name="mobile-web-app-capable" content="yes">
	<meta name="apple-mobile-web-app-capable" content="yes">
	<title>Big's Backyard Ultra, 10/19/2019-10/25/2019 : : my.race|result</title>
	<meta name="viewport" content="width=device-width, initial-scale=1.0">	
	<link rel="stylesheet" type="text/css" href="/general/style.css?v=1580065962">
	<link rel="stylesheet" href="/helper/nanoscroller.css">
	<link rel="stylesheet" type="text/css" href="index.css?v=1579592158">
	<link rel="shortcut icon" href="/graphics/favicon.png" type="image/png">
	<link rel="image_src

### Možnost 1: Developer tools - Network

Pokud nám jde o data jen z jedné webové stránky, můžeme pomocí nástrojů pro vývojáře zjistit, co javascript na stránce dělá. Zajímá nás, kde bere stránka data.

<div>
  <img src="static/bigdog2.png">
</div>

In [50]:
r = requests.get('https://my2.raceresult.com/RRPublish/data/list.php?eventid=139372&key=3a52cf488ad3dfe5c994f6b203e7c2e8&listname=Result+Lists%7CLap+Details&page=results&contest=0&r=all&l=0')

In [51]:
r.text

'{"list":{"ListName":"Result Lists|Lap Details","BottomPicture":"","BottomPictureShow":0,"ColumnHeadsFontBold":true,"ColumnHeadsFontItalic":false,"ColumnHeadsFontName":"","ColumnHeadsFontSize":12,"ColumnHeadsFontUnderlined":false,"ColumnHeadsColor":"","ColumnHeadsShow":2,"Columns":1,"ColumnSpacing":5,"CoverSheet":"","BackSheet":"","Design":"","DesignShow":2,"EveryOtherLineGray":false,"FontName":"Arial","FontSize":10,"FooterFontBold":false,"FooterFontItalic":false,"FooterFontName":"","FooterFontSize":10,"FooterFontUnderlined":false,"FooterColor":"","GrayLine":true,"HeadLine1":"[EventName]","HeadLine1FontBold":false,"HeadLine1FontItalic":false,"HeadLine1FontName":"","HeadLine1FontSize":16,"HeadLine1FontUnderlined":true,"HeadLine1Color":"","HeadLine1Show":2,"HeadLine2":"[Contestname]","HeadLine2FontBold":true,"HeadLine2FontItalic":false,"HeadLine2FontName":"","HeadLine2FontSize":16,"HeadLine2FontUnderlined":false,"HeadLine2Color":"","HeadLine2Show":2,"HeightBottomPicture":40,"LineColor":"

In [52]:
r.json()['data']

{'#1_1///Gavin Woody///40Laps': [['1', '1', '7:35:18', '55:19', '04:40'],
  ['1', '2', '8:35:23', '55:24', '04:35'],
  ['1', '3', '9:34:25', '54:26', '05:33'],
  ['1', '4', '10:35:09', '55:10', '04:49'],
  ['1', '5', '11:33:55', '53:56', '06:03'],
  ['1', '6', '12:34:23', '54:24', '05:35'],
  ['1', '7', '13:33:35', '53:36', '06:23'],
  ['1', '8', '14:33:57', '53:58', '06:01'],
  ['1', '9', '15:34:07', '54:08', '05:51'],
  ['1', '10', '16:34:39', '54:40', '05:19'],
  ['1', '11', '17:35:17', '55:18', '04:41'],
  ['1', '12', '18:34:22', '54:23', '05:36'],
  ['1', '13', '19:25:29', '45:30', '14:29'],
  ['1', '14', '20:28:55', '48:56', '11:03'],
  ['1', '15', '21:30:22', '50:23', '09:36'],
  ['1', '16', '22:27:05', '47:06', '12:53'],
  ['1', '17', '23:28:05', '48:06', '11:53'],
  ['1', '18', '24:28:35', '48:36', '11:23'],
  ['1', '19', '25:27:17', '47:18', '12:41'],
  ['1', '20', '26:28:12', '48:13', '11:46'],
  ['1', '21', '27:28:15', '48:16', '11:43'],
  ['1', '22', '28:26:35', '46:36', '

### Možnost 2: Headless browsers

**Selenium** je knihovna určená na automatizaci testování webových aplikací. Umožňuje spustit a ovládat prohlížeč, takže s ní můžete dělat prakticky všechno, co obvykle děláte na webu. Třeba automaticky vyplňovat formuláře nebo stahovat data.

In [53]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options  
#from selenium.webdriver.chrome.options import Options  
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [54]:
options = Options() 
options.headless = True

Nemáš-li chromedriver nebo geckodriver (pro Firefox), stáhneš je tady:
- [https://sites.google.com/a/chromium.org/chromedriver/home](https://sites.google.com/a/chromium.org/chromedriver/home)
- [https://github.com/mozilla/geckodriver/releases](https://github.com/mozilla/geckodriver/releases)

In [55]:
driver = webdriver.Firefox(options=options)
#driver = webdriver.Chrome(options=options)
driver.get('https://my3.raceresult.com/139372/#0_2C3B48')

<div>
  <img src="static/bigdog3.png">
</div>

In [56]:
try: # čekáme 5 s, jestli se tabulka natáhne 
    table = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'MainTable'))) 
except TimeoutException: 
    print("Time out!")

In [57]:
table

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="843f7c0c-3f7f-3648-8642-3e222823492a", element="253e470e-854c-ee4b-802d-67a4e5729831")>

In [58]:
txt = table.text

In [59]:
print(txt)

  #
Measurement
Lap Split
Rest Time
  1
Gavin Woody
40Laps
1 7:35:18 55:19 04:40  
2 8:35:23 55:24 04:35  
3 9:34:25 54:26 05:33  
4 10:35:09 55:10 04:49  
5 11:33:55 53:56 06:03  
6 12:34:23 54:24 05:35  
7 13:33:35 53:36 06:23  
8 14:33:57 53:58 06:01  
9 15:34:07 54:08 05:51  
10 16:34:39 54:40 05:19  
11 17:35:17 55:18 04:41  
12 18:34:22 54:23 05:36  
13 19:25:29 45:30 14:29  
14 20:28:55 48:56 11:03  
15 21:30:22 50:23 09:36  
16 22:27:05 47:06 12:53  
17 23:28:05 48:06 11:53  
18 24:28:35 48:36 11:23  
19 25:27:17 47:18 12:41  
20 26:28:12 48:13 11:46  
21 27:28:15 48:16 11:43  
22 28:26:35 46:36 13:23  
23 29:26:29 46:30 13:29  
24 30:27:35 47:36 12:23  
25 31:34:26 54:27 05:32  
26 32:32:42 52:43 07:16  
27 33:32:27 52:28 07:31  
28 34:34:26 54:27 05:32  
29 35:33:34 53:35 06:24  
30 36:33:37 53:38 06:21  
31 37:33:08 53:09 06:50  
32 38:34:02 54:03 05:56  
33 39:33:41 53:42 06:17  
34 40:33:59 54:00 05:59  
35 41:33:41 53:42 06:17  
36 42:36:38 56:39 03:20  
37 43:34:18 54:19

In [60]:
driver.close()
driver.quit()
